### Uploading Data from Wiki


In [1]:
# importing libraries 
import pandas as pd
import numpy as np
!pip install lxml
!pip install html5lib
!pip install BeautifulSoup4

!pip install geopy
from geopy.geocoders import Nominatim 
import requests # Handles requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library


print('Libraries Imported')

     |████████████████████████████████| 92kB 7.1MB/s eta 0:00:011
Libraries Imported


In [2]:
# read files into a pandas dataframe

from pandas.io.html import read_html 

# Get page and the table from page using class 'wikitable'

page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitables = read_html(page, attrs={"class": "wikitable"})

print('Data downloaded!')

Data downloaded!


In [3]:
wikitables[0].head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [26]:
#Define dataframe as df and replac "/" to "," 
df=wikitables[0]
df["Neighborhood"] = df["Neighborhood"].str.replace('/',",")
print(df.head())
print(df.dtypes)

  Postal code           Borough                Neighborhood
0         M1A      Not assigned                         NaN
1         M2A      Not assigned                         NaN
2         M3A        North York                   Parkwoods
3         M4A        North York            Victoria Village
4         M5A  Downtown Toronto  Regent Park , Harbourfront
Postal code     object
Borough         object
Neighborhood    object
dtype: object


In [27]:
# Delete rows with the value ['Not Assigned'] and make a new dataframe.
df1 = df[df.Borough != 'Not assigned']

df1.head(11)


,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern , Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill , Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [28]:
# Dataframe shape(rows and columns)
df1.shape

(103, 3)

In [29]:
# Reset index and rename first Column
df2=df1.reset_index(drop=True)
df2.rename(columns={"Postal code":'Postalcode'}, inplace=True)
df2.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [30]:
df2.shape

(103, 3)

In [31]:
# Load data from csv file and rename first column
path = 'http://cocl.us/Geospatial_data'
df3 = pd.read_csv(path)
df3.rename(columns={"Postal Code":'Postalcode'},inplace=True)
df3.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [32]:
# Merging the dataframe df2 and df3 based on first column
df4=pd.merge(df2,df3, on=["Postalcode"])

df4.head(12)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [33]:
toronto = df4[df4['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [34]:
toronto.shape

(39, 5)

In [35]:

import json # Handles JSON files

In [37]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="To_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [38]:
# create map of Toronto using latitude and longitude values

toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
         [lat, lng],
      radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

### Explore Neighborhoods
##### Explore first Neighborhood

In [39]:
toronto.loc[0, "Neighborhood"]

'Regent Park , Harbourfront'

Get Neighborhoods' Latitude and Longitude Values

In [40]:
neighborhood_latitude = toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park , Harbourfront are 43.6542599, -79.3606359.


#### Get top 50 venues that are in Regent Park / Harbourfront that are within 500 meters

In [41]:
CLIENT_ID = 'Y0A2AQDZKCRPGODASSROAZKIP0RMZDWC5SWDJWRE4Q3LMSE3' # your Foursquare ID
CLIENT_SECRET = 'JTZBM4N0GCEAOKTEFBM3JJ4N2DB550A1JWZ2TVYLF0NPTU52' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y0A2AQDZKCRPGODASSROAZKIP0RMZDWC5SWDJWRE4Q3LMSE3
CLIENT_SECRET:JTZBM4N0GCEAOKTEFBM3JJ4N2DB550A1JWZ2TVYLF0NPTU52


First Create get request URL and name the URL

In [42]:
LIMIT = 20
Radius = 500
Query = "food"
url='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,neighborhood_latitude,neighborhood_longitude, VERSION,Query,Radius,LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=Y0A2AQDZKCRPGODASSROAZKIP0RMZDWC5SWDJWRE4Q3LMSE3&client_secret=JTZBM4N0GCEAOKTEFBM3JJ4N2DB550A1JWZ2TVYLF0NPTU52&ll=43.6542599,-79.3606359&v=20180605&query=food&radius=500&limit=20'

Send get request and examine the result

In [43]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e99ea98bae9a2001be6d930'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'query': 'food',
  'totalResults': 22,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ae5b91ff964a520a6a121e3',
       'name': 'Morning Glory Cafe',
       'location': {'address': '457 King St. E',
        'crossStreet': 'Gilead Place',
        'lat': 43.653946942635294,
        'lng': -79.36114884214422,
        'labeledLatLngs'

In [44]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [45]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
1,Impact Kitchen,Restaurant,43.656369,-79.356980
2,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
3,Brick Street Bakery,Bakery,43.650574,-79.359539
4,Cluny Bistro & Boulangerie,French Restaurant,43.650565,-79.357843


Number of venues

In [46]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

20 venues were returned by Foursquare.


### Explore neighborhoods in Toronto

In [47]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### New dataframe as Toronto_Venues

In [48]:
Toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

Regent Park , Harbourfront
Queen's Park , Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
The Danforth West , Riverdale
Toronto Dominion Centre , Design Exchange
Brockton , Parkdale Village , Exhibition Place
India Bazaar , The Beaches West
Commerce Court , Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park , The Junction South
North Toronto West
The Annex , North Midtown , Yorkville
Parkdale , Roncesvalles
Davisville
University of Toronto , Harbord
Runnymede , Swansea
Moore Park , Summerhill East
Kensington Market , Chinatown , Grange Park
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Roseda

##### Dataframe Size

In [49]:
print(Toronto_venues.shape)
Toronto_venues.head()

(643, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park , Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


###### Counting Venues for each neighborhood

In [50]:
Toronto_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,20,20,20,20,20,20
"Brockton , Parkdale Village , Exhibition Place",20,20,20,20,20,20
Business reply mail Processing CentrE,14,14,14,14,14,14
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",18,18,18,18,18,18
Central Bay Street,20,20,20,20,20,20
Christie,18,18,18,18,18,18
Church and Wellesley,20,20,20,20,20,20
"Commerce Court , Victoria Hotel",20,20,20,20,20,20
Davisville,20,20,20,20,20,20


In [51]:
print('There are {} unique categories'.format(len(Toronto_venues['Venue Category'].unique())))

There are 166 unique categories


#### Analyze Data

In [52]:
# one hot encoding
toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,...,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
# New Dataframe size

toronto_onehot.shape

(643, 166)

In [54]:
toronto_grouped=toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,...,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,...,0.00,0.00,0.05,0.00,0.00,0.00,0.000000,0.05,0.0000,0.00
1,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.0000,0.00
2,Business reply mail Processing CentrE,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.0000,0.00
3,"CN Tower , King and Spadina , Railway Lands , ...",0.000000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.0000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.0000,0.00
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.0000,0.00
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.0000,0.00
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,...,0.00,0.00,0.05,0.00,0.05,0.00,0.000000,0.00,0.0000,0.00
7,"Commerce Court , Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0500,0.00,...,0.00,0.05,0.00,0.00,0.00,0.00,0.000000,0.00,0.0000,0.00
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,...,0.00,0.00,0.05,0.00,0.00,0.05,0.000000,0.00,0.0000,0.00
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.0000,0.00


In [55]:
toronto_grouped.shape

(39, 166)

##### Print each Neighborhood with top 5 venues

In [57]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0        Beer Bar  0.10
1  Farmers Market  0.10
2    Cocktail Bar  0.10
3          Museum  0.05
4        Fountain  0.05


----Brockton , Parkdale Village , Exhibition Place----
                venue  freq
0                Café  0.15
1         Coffee Shop  0.10
2      Breakfast Spot  0.10
3                 Gym  0.05
4  Italian Restaurant  0.05


----Business reply mail Processing CentrE----
                venue  freq
0          Skate Park  0.07
1       Auto Workshop  0.07
2         Pizza Place  0.07
3  Light Rail Station  0.07
4       Burrito Place  0.07


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3             Plane  0.06
4           Airport  0.06


----Central Bay Street----
                        venue  freq
0                 Coffee Shop  0.30
1          Ital

##### Change frequency data to Dataframe

In [58]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Top 10 venues of each Neighborhood

In [59]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Beer Bar,Cocktail Bar,Farmers Market,Liquor Store,Restaurant,Basketball Stadium,Seafood Restaurant,Bakery,Concert Hall,Park
1,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Breakfast Spot,Furniture / Home Store,Performing Arts Venue,Italian Restaurant,Climbing Gym,Bar,Bakery,Burrito Place
2,Business reply mail Processing CentrE,Garden,Gym / Fitness Center,Skate Park,Spa,Brewery,Farmers Market,Fast Food Restaurant,Pizza Place,Burrito Place,Auto Workshop
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Rental Car Location,Boat or Ferry,Sculpture Garden,Plane,Bar
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Sushi Restaurant,Park,Bubble Tea Shop,Japanese Restaurant,Ramen Restaurant,Modern European Restaurant,Bar


### Cluster Neighborhoods

Cluster Neighborhoods into 5 clusters using k-means

In [60]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1], dtype=int32)

New dataframe with cluster and top 10 venues in each neighborhood

In [61]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,0,Coffee Shop,Breakfast Spot,Distribution Center,Gym / Fitness Center,Dessert Shop,Chocolate Shop,Pub,Spa,Bakery,Farmers Market
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Hobby Shop,Mexican Restaurant,Park,Creperie,Burrito Place,Burger Joint,Diner
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Café,Coffee Shop,Steakhouse,Clothing Store,Music Venue,Burrito Place,Burger Joint,Comic Shop,Plaza,Ramen Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Gastropub,Coffee Shop,Gym,Church,Middle Eastern Restaurant,Cosmetics Shop,Café,Creperie,Japanese Restaurant,Italian Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Coffee Shop,Pub,Asian Restaurant,Wine Bar,Discount Store,Diner,Dessert Shop,Department Store


##### Visualize Clusters

In [62]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Checking Clusters

#### Cluster 1

In [76]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Breakfast Spot,Distribution Center,Gym / Fitness Center,Dessert Shop,Chocolate Shop,Pub,Spa,Bakery,Farmers Market
1,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Hobby Shop,Mexican Restaurant,Park,Creperie,Burrito Place,Burger Joint,Diner
4,East Toronto,0,Health Food Store,Trail,Coffee Shop,Pub,Asian Restaurant,Wine Bar,Discount Store,Diner,Dessert Shop,Department Store
6,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Sandwich Place,Sushi Restaurant,Park,Bubble Tea Shop,Japanese Restaurant,Ramen Restaurant,Modern European Restaurant,Bar
25,West Toronto,0,Gift Shop,Italian Restaurant,Breakfast Spot,Bookstore,Dog Run,Eastern European Restaurant,Bar,Restaurant,Dessert Shop,Cuban Restaurant


Coffee shop and restaurant is the the most common here

#### Cluster 2

In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1,Café,Coffee Shop,Steakhouse,Clothing Store,Music Venue,Burrito Place,Burger Joint,Comic Shop,Plaza,Ramen Restaurant
3,Downtown Toronto,1,Gastropub,Coffee Shop,Gym,Church,Middle Eastern Restaurant,Cosmetics Shop,Café,Creperie,Japanese Restaurant,Italian Restaurant
5,Downtown Toronto,1,Beer Bar,Cocktail Bar,Farmers Market,Liquor Store,Restaurant,Basketball Stadium,Seafood Restaurant,Bakery,Concert Hall,Park
7,Downtown Toronto,1,Grocery Store,Café,Park,Athletics & Sports,Diner,Restaurant,Baby Store,Candy Store,Italian Restaurant,Gas Station
8,Downtown Toronto,1,Asian Restaurant,Café,Opera House,Gym / Fitness Center,Speakeasy,Pizza Place,Concert Hall,Hotel,Steakhouse,Sushi Restaurant
9,West Toronto,1,Pizza Place,Pharmacy,Bakery,Gym / Fitness Center,Athletics & Sports,Middle Eastern Restaurant,Bar,Bank,Café,Park
10,Downtown Toronto,1,Café,Park,Plaza,New American Restaurant,Sporting Goods Shop,Performing Arts Venue,Lake,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant
11,West Toronto,1,Bar,Wine Bar,Record Shop,Art Gallery,Asian Restaurant,Beer Store,Brewery,Cocktail Bar,Cuban Restaurant,French Restaurant
12,East Toronto,1,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Cosmetics Shop,Pizza Place,Bubble Tea Shop,Pub,Brewery,Juice Bar,Restaurant
13,Downtown Toronto,1,Coffee Shop,Café,Steakhouse,Deli / Bodega,Gym / Fitness Center,Gym,Pizza Place,Japanese Restaurant,Art Gallery,American Restaurant


#### Cluster 3

In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,2,Restaurant,Playground,Wine Bar,Cosmetics Shop,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant


#### Cluster 4

In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Swim School,Park,Bus Line,Concert Hall,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant
33,Downtown Toronto,3,Park,Playground,Trail,Concert Hall,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant


#### Cluster 5

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4,Garden,Home Service,Wine Bar,Creperie,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio


# Thank you!!!!